In [ ]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2022,16)
from solution import *

In [ ]:
input_text = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""

In [ ]:

# floyd-warshall-algorithm
for valve_id, valve in parse_input(input_text.splitlines()).items():
        valves[valve_id] =  valve

for valve_id, valve in valves.items():
    print(f"{valve_id}: {valve}")
# valve_names = list(sorted(valves.keys()))
# graph = [[float('infinity') for _ in range(len(valve_names))] for _ in range(len(valve_names))]
# for idx, valve_id in enumerate(valve_names):
#     for idx2, next_valve_id in enumerate(valve_names):
#         if valve_id == next_valve_id:
#             graph[idx][idx2] = 0
#         if next_valve_id in valves[valve_id]['tunnels']:
#             graph[idx][idx2] = 1
# graph = floyd_warshall(graph)
         
# print()
# for row in graph:
#     print(row)

get_ordered_next_hops('AA')

In [ ]:
for valve_id, valve in parse_input(input_text.splitlines()).items():
        valves[valve_id] =  valve
working_valves = {}
for valve_id, valve in valves.items():
    if valve["flow_rate"] > 0 or valve_id == 'AA':
        working_valves[valve_id] = valve

for valve_id, valve in working_valves.items():
    valve["next_hops"] = []
    for next_valve in working_valves:
        if valve_id == next_valve or next_valve == 'AA':
            continue
        valve["next_hops"].append(next_valve)

working_valves['DONE'] = {
    'next_hops': ['DONE']
}
print(working_valves)
print(max_pressure_relief(working_valves, time_limit=30, workers=1))
print(max_pressure_relief(working_valves, time_limit=26, workers=2))





In [ ]:
import itertools

a=('AA','BB','DD')
b=('CC','BB','EE')

for combo in itertools.product(a,b):
    print(combo)


In [ ]:
for combo in itertools.product(a):
    print(combo)

In [ ]:
# Python3 Program for Floyd Warshall Algorithm

# Number of vertices in the graph
V = 4

# Define infinity as the large
# enough value. This value will be
# used for vertices not connected to each other
INF = 99999

# Solves all pair shortest path
# via Floyd Warshall Algorithm


def floydWarshall(graph):
    """ dist[][] will be the output 
       matrix that will finally
        have the shortest distances 
        between every pair of vertices """
    """ initializing the solution matrix 
    same as input graph matrix
    OR we can say that the initial 
    values of shortest distances
    are based on shortest paths considering no 
    intermediate vertices """

    dist = list(map(lambda i: list(map(lambda j: j, i)), graph))

    """ Add all vertices one by one 
    to the set of intermediate
     vertices.
     ---> Before start of an iteration, 
     we have shortest distances
     between all pairs of vertices 
     such that the shortest
     distances consider only the 
     vertices in the set 
    {0, 1, 2, .. k-1} as intermediate vertices.
      ----> After the end of a 
      iteration, vertex no. k is
     added to the set of intermediate 
     vertices and the 
    set becomes {0, 1, 2, .. k}
    """
    for k in range(V):

        # pick all vertices as source one by one
        for i in range(V):

            # Pick all vertices as destination for the
            # above picked source
            for j in range(V):

                # If vertex k is on the shortest path from
                # i to j, then update the value of dist[i][j]
                dist[i][j] = min(dist[i][j],
                                 dist[i][k] + dist[k][j]
                                 )
    printSolution(dist)
    print(dist)


# A utility function to print the solution
def printSolution(dist):
    print("Following matrix shows the shortest distances\
 between every pair of vertices")
    for i in range(V):
        for j in range(V):
            if(dist[i][j] == INF):
                print("%7s" % ("INF"), end=" ")
            else:
                print("%7d\t" % (dist[i][j]), end=' ')
            if j == V-1:
                print()


# Driver's code
if __name__ == "__main__":
    """
                10
           (0)------->(3)
            |         /|\
          5 |          |
            |          | 1
           \|/         |
           (1)------->(2)
                3           """
    graph = [[0, 5, INF, 10],
             [INF, 0, 3, INF],
             [INF, INF, 0,   1],
             [INF, INF, INF, 0]
             ]
    # Function call
    floydWarshall(graph)
# This code is contributed by Mythri J L

In [ ]:
import re
import time
from itertools import combinations
from collections import deque, defaultdict

start_time = time.time()

valves = {}
flow_rates = {}
file = open("input.txt")
# for line in input_text.splitlines():
for line in file:
  tokens = line.strip().split(' ')

  valve = tokens[1]
  flow_rate = int(re.findall(r'\d+', tokens[4])[0])
  neighbors = set([node for node in ''.join(tokens[9:]).split(',')])

  valves[valve] = neighbors
  flow_rates[valve] = flow_rate

# memoization helper
def memoize(fn):
  memo = {}

  def memozied(*args):
    if args in memo:
      return memo[args]
    else:
      result = fn(*args)
      memo[args] = result
      return result

  return memozied

# yield all possible partitions of a list into k subsets
def subsets_k(collection, k): yield from partition_k(collection, k, k)

def partition_k(collection, min, k):
  if len(collection) == 1:
    yield [ collection ]
    return

  first = collection[0]
  for smaller in partition_k(collection[1:], min - 1, k):
    if len(smaller) > k: continue

    if len(smaller) >= min:
      for n, subset in enumerate(smaller):
        yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]

    if len(smaller) < k: yield [ [ first ] ] + smaller

# BFS the valve graph to return the shortest path length
# betweent two valves
@memoize
def shorest_path_length(start, end):
  q = deque([[start]])
  visited = set()

  if start == end:
    return len([start]) - 1

  while len(q) > 0:
    path = q.popleft()
    node = path[-1]

    if node not in visited:
      neighbors = valves[node]
      
      for neighbor in neighbors:
        new_path = list(path)
        new_path.append(neighbor)
        q.append(new_path)

        if neighbor == end:
          return len(new_path) - 1
      
      visited.add(node)
  
  return None

@memoize
def valve_choose_r(serialized_valves, r):
  return combinations(serialized_valves.split(','), r)

# DFS of every possible order we could open the valves in
#
# cut down on the search space by eliminating valves with flow rate of 0
# we will never stop at those valves to open them
def should_try_to_open_valve(valve):
  return flow_rates[valve] != 0

valves_to_open = list(filter(should_try_to_open_valve, valves.keys()))

start_location = 'AA'
time_limit = 26

maximum_pressure = 0
pressure_by_path = defaultdict(lambda: 0)

# a stack of our current branches - [path], minutes_elapsed, { valve: <minute opened> }
s = [[[start_location], 0, {}]]

while len(s):
  path, minutes_elapsed, open_valves = s.pop()
  current_valve = path[-1]

  pressure_released = 0

  for valve, minute_opened in open_valves.items():
    minutes_opened = max(time_limit - minute_opened, 0)
    pressure_released += flow_rates[valve] * minutes_opened

  maximum_pressure = max(maximum_pressure, pressure_released)

  # sort the valves lexically and use it as a key
  # so we can determine which ordering of these valves
  # nets the greatest pressure released
  path_key = ','.join(sorted(path))
  pressure_by_path[path_key] = max(pressure_by_path[path_key], pressure_released)

  # if we've run out of time, abandon this "branch"
  if minutes_elapsed >= time_limit or len(path) == len(valves_to_open) + 1:
    # because we may not have opened all of the valves, all other possible ways
    # of adding the remaining valves need to be added to the lookup
    # but the pressure will be the same
    remaining_valves = [valve for valve in valves_to_open if valve not in path]

    for r in range(1, len(remaining_valves)):
      for combination in valve_choose_r(','.join(sorted(remaining_valves)), r):
        key = ','.join(sorted(path + list(combination)))
        pressure_by_path[key] = max(pressure_by_path[key], pressure_released)

    continue
  else:
    for next_valve in valves_to_open:
      if next_valve not in open_valves.keys():
        # each edge in the graph takes 1 minute to cross
        # so our travel time to the next valve we're trying to open
        # is the length of the shortest path
        # taking anything longer than the shortest possible path
        # would obviously be sub-optimal
        travel_time = shorest_path_length(current_valve, next_valve)

        # plus we'll stop at the valve for an additional minute to open it
        time_to_open_valve = 1

        new_minutes_elapsed = minutes_elapsed + travel_time + time_to_open_valve

        new_open_valves = open_valves.copy()
        new_open_valves[next_valve] = new_minutes_elapsed

        new_path = list(path)
        new_path.append(next_valve)

        s.append([new_path, new_minutes_elapsed, new_open_valves])

# check if we can increase the pressure by training the elephants
#
# generate all of the possible ways to split the valves
# between me and the elephants, it doesn't matter who takes which subset
#
# https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind
for i, subsets in enumerate(subsets_k(valves_to_open, 2)):
  my_pressure = pressure_by_path[','.join(sorted(['AA'] + subsets[0]))]
  elephant_pressure = pressure_by_path[','.join(sorted(['AA'] + subsets[1]))]
  if (my_pressure + elephant_pressure) > maximum_pressure:
    print(','.join(sorted(['AA'] + subsets[0])), ','.join(sorted(['AA'] + subsets[1])))
  maximum_pressure = max(maximum_pressure, my_pressure + elephant_pressure)

print(maximum_pressure)
print("%s seconds" % (time.time() - start_time))

In [ ]:
import itertools
for idx, combo in enumerate(itertools.product(('MD', 'YW', 'FS', 'TX', 'SS', 'IK', 'JE', 'CR', 'KI', 'YB', 'IT'), ('FS', 'JE', 'MD', 'TX', 'YW', 'IK', 'SS', 'CR', 'YB', 'IT', 'KI'))):
    print(idx, combo)

In [ ]:
current=((0,),(1,),(2,),(3,),(4,),(5))
print(current[1:4])

In [ ]:
for idx, subset in enumerate(subsets_k(valves_to_open, 2)):
    print(idx, subset)